# GP-ConvCNP Examples

This notebook will allow you to explore our proposed GP-ConvCNP as well as all the baselines. You will need to install the `neuralprocess` package contained within the ZIP file you received. For instructions on how to do that, please consult the `README.md`.

Below, we will first construct data generators that produce functions like the ones used in the paper. We then load pretrained models and visualize predictions similar to the figures found in our paper.

## Imports

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from neuralprocess.data import GaussianProcessGenerator, StepFunctionGenerator, gp
from neuralprocess.model import pretrained

## Data Generators

These generators are constructed to match what the models were trained on (except num_context and num_target, which are fixed). Feel free to change parameters to see how well the methods handle those changes, both in distribution (e.g. num_context from 5 to 50) and out of distribution (e.g. different kernel lengthscale)

In [2]:
base_kwargs = dict(
    batch_size=1,
    x_range=(-3, 3),
    num_context=20,
    num_target=200,
    output_noise=0.,
)

generators = {
    "matern": GaussianProcessGenerator(
        **base_kwargs,
        kernel_type=gp.Matern52Kernel,
        kernel_kwargs=dict(lengthscale=0.5)
    ),
    "wp": GaussianProcessGenerator(
        **base_kwargs,
        kernel_type=gp.WeaklyPeriodicKernel,
        kernel_kwargs=dict()
    ),
    "step": StepFunctionGenerator(
        **base_kwargs,
        y_range=(-3, 3),
        number_of_steps=(3, 10),
        min_step_width=0.1,
        min_step_height=0.1
    )
}

## Pretrained Models

The models are very small (NP ~1.3MB, ANP ~2.1MB, ConvCNP & GP-ConvCNP ~207kB), so we can load them all at once.

In [2]:
NP_models = {}
ANP_models = {}
ConvCNP_models = {}
GPConvCNP_models = {}
for function_type in ("matern", "wp", "step"):
    NP_models[function_type] = pretrained.NeuralProcess(function_type)
    ANP_models[function_type] = pretrained.AttentiveNeuralProcess(function_type)
    ConvCNP_models[function_type] = pretrained.ConvCNP(function_type, use_gp=False)
    GPConvCNP_models[function_type] = pretrained.ConvCNP(function_type, use_gp=True)